In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/05 15:55:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!curl -L https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz | gzip -d > fhvhv_tripdata_2021-01.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  123M  100  123M    0     0  38.2M      0  0:00:03  0:00:03 --:--:-- 53.7M


24/03/05 15:56:14 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [4]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [6]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [7]:
df.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
+-----------------+--------------------+-------------------+-------------------+

In [8]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [9]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [10]:
import pandas as pd

In [11]:
df_pandas = pd.read_csv('head.csv')

In [12]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [13]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

Integer - 4 bytes
Long - 8 bytes

In [14]:
from pyspark.sql import types

In [15]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [24]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [25]:
df = df.repartition(24)

In [26]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

24/03/05 16:21:11 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/05 16:21:12 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/05 16:21:13 WARN MemoryManager: Total allocation exceeds 95.00% (1’020’054’720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [27]:
df = spark.read.parquet('fhvhv/2021/01/')

In [28]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [31]:
from pyspark.sql import functions as F

In [32]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02883|2021-01-02 20:27:02|2021-01-02 20:35:57|         249|         107|   NULL|
|           HV0003|              B02835|2021-01-02 16:38:39|2021-01-02 16:49:56|         225|          62|   NULL|
|           HV0003|              B02765|2021-01-03 15:53:25|2021-01-03 15:57:36|         123|         123|   NULL|
|           HV0005|              B02510|2021-01-02 00:46:07|2021-01-02 00:59:00|         143|          24|   NULL|
|           HV0003|              B02887|2021-01-02 18:27:36|2021-01-02 19:07:42|         132|         265|   NULL|
|           HV0005|              B02510|2021-01-02 14:24:32|2021-01-02 14:35:49|

In [33]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [34]:
crazy_stuff('B02884')

's/b44'

In [36]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [37]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  a/b43| 2021-01-02|  2021-01-02|         249|         107|
|  s/b13| 2021-01-02|  2021-01-02|         225|          62|
|  s/acd| 2021-01-03|  2021-01-03|         123|         123|
|  e/9ce| 2021-01-02|  2021-01-02|         143|          24|
|  e/b47| 2021-01-02|  2021-01-02|         132|         265|
|  e/9ce| 2021-01-02|  2021-01-02|          48|         163|
|  s/b13| 2021-01-01|  2021-01-01|         255|          79|
|  e/b3e| 2021-01-03|  2021-01-03|         243|         119|
|  e/9ce| 2021-01-03|  2021-01-03|         244|         244|
|  e/b3b| 2021-01-01|  2021-01-01|         181|         125|
|  s/acd| 2021-01-01|  2021-01-01|          92|          42|
|  e/acc| 2021-01-04|  2021-01-04|         144|          68|
|  e/b38| 2021-01-05|  2021-01-05|          17|         181|
|  a/a7a| 2021-01-02|  2

24/03/05 17:33:27 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 878341 ms exceeds timeout 120000 ms
24/03/05 17:33:27 WARN SparkContext: Killing executors is not supported by current scheduler.
24/03/05 17:33:32 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

In [30]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003') \
  .show()


+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-02 20:27:02|2021-01-02 20:35:57|         249|         107|
|2021-01-02 16:38:39|2021-01-02 16:49:56|         225|          62|
|2021-01-03 15:53:25|2021-01-03 15:57:36|         123|         123|
|2021-01-02 18:27:36|2021-01-02 19:07:42|         132|         265|
|2021-01-01 01:26:50|2021-01-01 01:40:27|         255|          79|
|2021-01-03 11:47:58|2021-01-03 11:55:33|         243|         119|
|2021-01-01 10:32:57|2021-01-01 10:52:13|         181|         125|
|2021-01-01 03:21:44|2021-01-01 04:04:56|          92|          42|
|2021-01-04 14:32:22|2021-01-04 14:47:36|         144|          68|
|2021-01-05 10:12:17|2021-01-05 10:31:43|          17|         181|
|2021-01-02 19:48:31|2021-01-02 19:56:13|         228|         227|
|2021-01-04 20:56:43|2021-01-04 21:15:38|       

In [50]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag

HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,

HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,

HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,

HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,

HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,

HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,

HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,

HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,

HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,



In [42]:
df = pd.read_csv('/Users/amaliatemneanu/Documents/Projects/data-engineering-zoomcamp-assignments/5.1_batch/data/raw/fhv/2019/10/fhv_tripdata_2019_10.csv.gz')

In [43]:
df

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2019-10-01 00:23:00,2019-10-01 00:35:00,264.0,264.0,NaN,B00009
1,B00013,2019-10-01 00:11:29,2019-10-01 00:13:22,264.0,264.0,NaN,B00013
2,B00014,2019-10-01 00:11:43,2019-10-01 00:37:20,264.0,264.0,NaN,B00014
3,B00014,2019-10-01 00:56:29,2019-10-01 00:57:47,264.0,264.0,NaN,B00014
4,B00014,2019-10-01 00:23:09,2019-10-01 00:28:27,264.0,264.0,NaN,B00014
...,...,...,...,...,...,...,...
1897488,B03160,2019-10-31 23:38:00,2019-10-31 23:48:00,242.0,81.0,NaN,B02887
1897489,B03160,2019-10-31 23:11:00,2019-10-31 23:43:00,161.0,28.0,NaN,B02883
1897490,B03160,2019-10-31 23:13:00,2019-10-31 23:41:00,168.0,215.0,NaN,B02883
1897491,B03186,2019-10-31 23:02:32,2019-10-31 23:09:53,264.0,119.0,NaN,B03186
